In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/hice1/dlee906/.conda/envs/layer_skip_1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
model_name = "facebook/opt-2.7b"
model = AutoModelForCausalLM.from_pretrained("facebook/layerskip-llama2-7B")
print(model)


The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


Loading checkpoint shards: 100%|██████████| 3/3 [00:10<00:00,  3.57s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (no

In [15]:
from transformers import AutoModelForCausalLM
import torch
import os

# Function for pruning the model
def prune_model(model_name, prune_ratio, save_path):
    """
    Prune the model layers and save the pruned model.

    Parameters:
    - model_name: str, name or path of the model to prune (e.g., "facebook/layerskip-llama2-7B")
    - prune_ratio: float, the percentage of layers to keep (e.g., 0.5 for 50% of layers)
    - save_path: str, directory to save the pruned model
    """
    # Load the model
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Get the number of layers and prune
    num_layers = len(model.model.layers)
    keep_layers = sorted(torch.randperm(num_layers)[:int(num_layers * (1 - prune_ratio))].tolist())
    
    print(f"Keeping layers: {keep_layers}")

    # Prune layers
    model.model.layers = torch.nn.ModuleList([model.model.layers[i] for i in keep_layers])

    # Save the pruned model
    model.save_pretrained(save_path)
    print(f"Pruned model saved to {save_path}")

# Example usage - you can run this cell directly in Jupyter
model_name = "facebook/layerskip-llama2-7B"
prune_ratio = 0.5  # 50% pruning
save_path = "./pruned_layerskip"

# Run the pruning
prune_model(model_name, prune_ratio, save_path)


Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.46s/it]


Keeping layers: [2, 3, 4, 5, 7, 9, 10, 11, 12, 14, 18, 20, 21, 23, 27, 28]
Pruned model saved to ./pruned_layerskip


In [14]:
import ipywidgets as widgets
from IPython.display import display

prune_ratio_slider = widgets.FloatSlider(value=0.5, min=0.1, max=0.9, step=0.1, description='Prune Ratio:')
display(prune_ratio_slider)

# Later in the cell:
prune_model("facebook/layerskip-llama2-7B", prune_ratio_slider.value, "./pruned_layerskip")


ModuleNotFoundError: No module named 'ipywidgets'